In [1]:
from bs4 import BeautifulSoup as Soup, Tag
from os import listdir
from pathlib import Path
from itertools import takewhile,dropwhile
from tg.projects.retell.retell_utils import ChapterInfo,BookInfo
from typing import *
import re

In [2]:
path_plio_books = Path("../fb2_to_md/eng_martin/fb2")
plio_books = listdir(path_plio_books)

In [3]:
plio_books

['Martin-A Game of Thrones.fb2',
 'Martin-A Storm of Swords.fb2',
 'Martin-A Feast for Crows.fb2',
 'Martin-A Dance with Dragons.fb2',
 'Martin-A Clash of Kings.fb2']

In [4]:
def compile_chapter(section: Tag):
    title = title_tag.get_text(strip=True) if (
        title_tag := section.find('title')
    ) else ''
    rows = ''
    if hasattr(title_tag, 'find_next_siblings'):
        rows = '\n'.join(
            [p.get_text(strip=True) for p in title_tag.find_next_siblings('p')]
        )
    return title, rows


books: List[BookInfo] = []

for book in plio_books:
    book_name = book.split('.')[0].split('-')[-1]
    book_info = BookInfo(book_name, [])
    with open(path_plio_books / book, 'r', encoding='utf-8') as xml:
        soup = Soup(xml.read(), 'lxml')
        book_info.chapters = [
            ChapterInfo(name=chapter_info[0], retell=chapter_info[1])
            for chapter_info in
                takewhile(
                    lambda chapter_data: not chapter_data[0].lower().startswith('appendix') and
                                         not chapter_data[0].lower().startswith('meanwhile'),
                    dropwhile(
                        lambda chapter_data: not chapter_data[0].lower().startswith('prologue'),
                        map(compile_chapter, soup.find_all('section'))
                    )
                )
        ]
        books.append(book_info)

/home/mixailkys/anaconda3/envs/grammar_ru/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
book_data_path = Path("../fb2_to_md/eng_martin/md")

In [6]:
def eng_write_game_o_trones_md(game_o_trones):
    for book in game_o_trones:
        with open(Path(book_data_path / f'Martin-{book.name.replace("_", " ")}-book.md'), 'w') as file:
            for i,chapter in enumerate(book.chapters):
                retell = re.sub("[\(\[].*?[\)\]]", "", chapter.retell)
                retell = re.sub(r"[(\[{})\]]", "", retell)
                file.write(f"##{chapter.name.capitalize()}-Chapter_{i}\n\n{retell}\n\n")

In [7]:
eng_write_game_o_trones_md(books)